## **Setting up Enviroment**

In [ ]:

#install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
#'''
file_loc = './sample_data/california_housing_train.csv'
#'''

In [ ]:
# Point Colaboratory to your Google Drive
# Mount Dataset from GDrive
from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

### **Import Necessary Libraries**

In [ ]:
import librosa 
import librosa.display
!pip install SoundFile
import soundfile 
import os, glob
from time import time
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neural_network import MLPClassifier 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer, confusion_matrix, classification_report

## **Feature Extraction**

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel, contrast, tonnetz):
    with soundfile.SoundFile(file_name) as sound_file:
        X,sr = librosa.core.load(file_name, mono=True, dtype="float32")
        #X = sound_file.read(always_2d=True, dtype="float32")
        sample_rate=sound_file.samplerate
        #sound_file.channels = 1
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [ ]:
#emotions as suggested in the RAVDESS dataset filenames
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#classes we want to classify the emotions in
observed_emotions=['03', '04', '05', '08']

## **Load Dataset**

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=file_name.split("-")[2]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True)
        x.append(feature)
        y.append(emotion)
        #tvs = TrainValidationSplit(trainRatio=0.5)
#    return tvs
    return x,y


#import IPython.display as ipd
#import matplotlib.pyplot as plt

#ipd.Audio('/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_01/03-01-01-01-01-01-01.wav')

#data, sampling_rate = librosa.load('/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_01/03-01-01-01-01-01-01.wav')
#plt.figure(figsize=(12, 4))
#librosa.display.waveplot(data, sr=sampling_rate)


## **Split Dataset**

In [ ]:
x,y=load_data(test_size=0.25)

In [ ]:
z = train_test_split(np.array(x), y, test_size=0.25, random_state=9)
x_train, x_test, y_train, y_test = z

In [ ]:
print("No. of training samples:", x_train.shape[0]," No. of testing samples:", x_test.shape[0])
#print(type(x_train))
x=np.array(x)
print(x.shape)
#print(type(x))
#print(type(y))
#print(type(y_train))

No. of training samples: 504  No. of testing samples: 168
(672, 193)


## **Feature Extraction**

In [ ]:
print(f'No. of Features extracted: {x_train.shape[1]}')

No. of Features extracted: 193


In [ ]:
'''
!pip install pydub
from pydub import AudioSegment
for file in glob.glob("/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_*//*.wav"):
  wav_file = AudioSegment.from_file(file = file, format="wav")
  #print(wav_file.channels)
  wav_file = wav_file.set_channels(1)

  wav_file.export(path.join(content/gdrive/My Drive/Colab Datasets/Dataset), "file.wav", format="wav")
'''

'\n!pip install pydub\nfrom pydub import AudioSegment\nfor file in glob.glob("/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_*//*.wav"):\n  wav_file = AudioSegment.from_file(file = file, format="wav")\n  #print(wav_file.channels)\n  wav_file = wav_file.set_channels(1)\n\n  wav_file.export(path.join(content/gdrive/My Drive/Colab Datasets/Dataset), "file.wav", format="wav")\n'

##**Feature Selection**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
s = StandardScaler()
s.fit(x_train)
#print(X_train)
x_train_scaled = s.transform(x_train)
x_test_scaled = s.transform(x_test)

#Perform PCA

pca = PCA()
x_train = pca.fit_transform(x_train_scaled)
x_test = pca.transform(x_test_scaled)

explained_variance = pca.explained_variance_ratio_
print("explained_variance ",explained_variance)

explained_variance  [2.57389090e-01 8.47241531e-02 5.26199794e-02 4.81547657e-02
 4.06468677e-02 3.24882054e-02 2.97072195e-02 2.75447289e-02
 2.54223608e-02 2.43796607e-02 2.07316568e-02 1.85745279e-02
 1.70296119e-02 1.64438094e-02 1.52339500e-02 1.34068550e-02
 1.20916937e-02 1.17783281e-02 1.13203735e-02 1.09646602e-02
 9.84151914e-03 9.64557740e-03 9.29649416e-03 8.64190589e-03
 8.34043009e-03 7.47071412e-03 7.32986401e-03 7.03032355e-03
 6.75405778e-03 6.25675978e-03 5.91094808e-03 5.71195334e-03
 5.28144432e-03 5.02442493e-03 4.82750334e-03 4.48418830e-03
 4.41535773e-03 4.26100166e-03 4.20828876e-03 3.90664412e-03
 3.72084979e-03 3.67124312e-03 3.39986735e-03 3.35297916e-03
 3.26704909e-03 3.12117314e-03 2.93796083e-03 2.84675284e-03
 2.74318832e-03 2.64450261e-03 2.49915188e-03 2.40538139e-03
 2.30968367e-03 2.21805823e-03 2.11281936e-03 2.08987721e-03
 2.02546724e-03 2.00266328e-03 1.91190332e-03 1.84987505e-03
 1.76765884e-03 1.71352784e-03 1.63462056e-03 1.54060951e-03
 1.4

# **Classification**



In [ ]:
scoring = {'accuracy':make_scorer(accuracy_score),
           'precision':make_scorer(precision_score, average='micro'),
           'recall':make_scorer(recall_score, average='micro'),
           'f1_score':make_scorer(f1_score, average='micro')}

In [ ]:
#Initialize all the ml classifiers

#Multilayer Perceptron Classifier
m1_model=MLPClassifier(alpha=0.005, batch_size=256, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Support Vector Classifier
s1_model=SVC(C=0.001, kernel='poly', gamma=0.001)

#Random Forest Classifier
r1_model=RandomForestClassifier(max_depth=7, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=40)

#Decision Tree Classifier
d1_model=DecisionTreeClassifier(criterion='entropy', max_depth=7, max_features=None, min_samples_leaf=1, min_samples_split=2)

In [ ]:
'''
#Classifier evaluation function

def classifier_evaluation(X, y, folds):

  X : features
  y : target
  folds : no. of cross-validation folds
  
  
  m1 = cross_validate(m1_model, X, y, cv=folds, scoring=scoring)
  s1 = cross_validate(s1_model, X, y, cv=folds, scoring=scoring)
  r1 = cross_validate(r1_model, X, y, cv=folds, scoring=scoring)
  d1 = cross_validate(d1_model, X, y, cv=folds, scoring=scoring)

  #Data frame for score table

  classifier_score_table = pd.DataFrame({'Multilayer Perceptron Classifier':[m1['test_accuracy'].mean(),
                                                                             m1['test_precision'].mean(),
                                                                             m1['test_recall'].mean(),
                                                                             m1['test_f1_score'].mean()],
                                         
                                         'Support Vector Classifier':[s1['test_accuracy'].mean(),
                                                                      s1['test_precision'].mean(),
                                                                      s1['test_recall'].mean(),
                                                                      s1['test_f1_score'].mean()],
                                         
                                         'Random Forest Classifier':[r1['test_accuracy'].mean(),
                                                                     r1['test_precision'].mean(),
                                                                     r1['test_recall'].mean(),
                                                                     r1['test_f1_score'].mean()],
                                         
                                         'Decision Tree Classifier':[d1['test_accuracy'].mean(),
                                                                     d1['test_precision'].mean(),
                                                                     d1['test_recall'].mean(),
                                                                     d1['test_f1_score'].mean()]},
                                        
                                          index=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

  #Best Score Column
  classifier_score_table['Best Score']=classifier_score_table.idxmax(axis=1)

  return(classifier_score_table)

classifier_evaluation(x_train, y_train, 5)
'''

"\n#Classifier evaluation function\n\ndef classifier_evaluation(X, y, folds):\n\n  X : features\n  y : target\n  folds : no. of cross-validation folds\n  \n  \n  m1 = cross_validate(m1_model, X, y, cv=folds, scoring=scoring)\n  s1 = cross_validate(s1_model, X, y, cv=folds, scoring=scoring)\n  r1 = cross_validate(r1_model, X, y, cv=folds, scoring=scoring)\n  d1 = cross_validate(d1_model, X, y, cv=folds, scoring=scoring)\n\n  #Data frame for score table\n\n  classifier_score_table = pd.DataFrame({'Multilayer Perceptron Classifier':[m1['test_accuracy'].mean(),\n                                                                             m1['test_precision'].mean(),\n                                                                             m1['test_recall'].mean(),\n                                                                             m1['test_f1_score'].mean()],\n                                         \n                                         'Support Vector Classifier':[s1['

### **Multi Layer Perceptron**

In [ ]:

#initialize the model
m1=MLPClassifier(alpha=0.005, batch_size=256, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Train the model
m1.fit(x_train,y_train)

#Predict for the test set
m1y_pred=m1.predict(x_test)

#Calculate the accuracy of our model
acc_m1=accuracy_score(y_true=y_test, y_pred=m1y_pred)
#f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='weighted')
f1_m1=f1_score(y_true=y_test, y_pred=m1y_pred, average='weighted')
wP_m1=precision_score(y_true=y_test, y_pred=m1y_pred, average='weighted')
wR_m1=recall_score(y_true=y_test, y_pred=m1y_pred, average='weighted')
'''
#Calculate fbeta score
f_m1=fbeta_score(y_test, m1y_pred, beta=0.5, average='micro')
'''
#Calculate confusion matrix
#cm1=confusion_matrix(y_test, m1y_pred, labels=observed_emotions, average='weighted').astype(np.float32)

#Print the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_m1*100))
print("d1 for MLP : {:.2f}%".format(f1_m1*100))
print("wp for MLP : {:.2f}%".format(wP_m1*100))
print("wr for MLP : {:.2f}%".format(wR_m1*100))
'''
#Display confusion matrix
plt.imshow(cm1, cmap="binary")
plt.show()
'''

Accuracy for MLP : 67.26%
d1 for MLP : 67.36%
wp for MLP : 68.93%
wr for MLP : 67.26%


'\n#Display confusion matrix\nplt.imshow(cm1, cmap="binary")\nplt.show()\n'

### **Support Vector Machine**

In [ ]:
'''
#initialize the model
s1=SVC(C=0.001, kernel='poly', gamma=0.001)

#Train the model
s1.fit(x_train, y_train)

#Predict for the test set
s1y_pred=s1.predict(x_test)

#Calculate the accuracy of our model
acc_s1=accuracy_score(y_true=y_test, y_pred=s1y_pred)

#Calculate fbeta score
f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='micro')

#Calculate confusion matrix
cm2=confusion_matrix(y_test, s1y_pred, labels=observed_emotions).astype(np.float32)

#Print the accuracy
print("Accuracy for SVC : {:.2f}%".format(acc_s1*100))

#Display confusion matrix
plt.imshow(cm2, cmap="binary")
plt.show()
'''

'\n#initialize the model\ns1=SVC(C=0.001, kernel=\'poly\', gamma=0.001)\n\n#Train the model\ns1.fit(x_train, y_train)\n\n#Predict for the test set\ns1y_pred=s1.predict(x_test)\n\n#Calculate the accuracy of our model\nacc_s1=accuracy_score(y_true=y_test, y_pred=s1y_pred)\n\n#Calculate fbeta score\nf_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average=\'micro\')\n\n#Calculate confusion matrix\ncm2=confusion_matrix(y_test, s1y_pred, labels=observed_emotions).astype(np.float32)\n\n#Print the accuracy\nprint("Accuracy for SVC : {:.2f}%".format(acc_s1*100))\n\n#Display confusion matrix\nplt.imshow(cm2, cmap="binary")\nplt.show()\n'

### **Random Forest Classifier**

In [ ]:

#initialize the model
r1=RandomForestClassifier(max_depth=7, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=40)

#Train the model
r1.fit(x_train, y_train)

#Predict for the test set
r1y_pred=r1.predict(x_test)

#Calculate the accuracy of our model

acc_r1=accuracy_score(y_true=y_test, y_pred=r1y_pred)
#f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='weighted')
f1_r1=f1_score(y_true=y_test, y_pred=r1y_pred, average='weighted')
wP_r1=precision_score(y_true=y_test, y_pred=r1y_pred, average='weighted')
wR_r1=recall_score(y_true=y_test, y_pred=r1y_pred, average='weighted')
#Calculate fbeta score
#f_r1=fbeta_score(y_test, r1y_pred, beta=0.5, average='micro')

#Calculate confusion matrix
#cm3=confusion_matrix(y_test, r1y_pred, labels=observed_emotions).astype(np.float32)

#Print the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_r1*100))
print("d1 for MLP : {:.2f}%".format(f1_r1*100))
print("wp for MLP : {:.2f}%".format(wP_r1*100))
print("wr for MLP : {:.2f}%".format(wR_r1*100))
'''
plt.imshow(cm3, cmap="binary")
plt.show()
'''

Accuracy for MLP : 57.14%
d1 for MLP : 57.12%
wp for MLP : 57.16%
wr for MLP : 57.14%


'\nplt.imshow(cm3, cmap="binary")\nplt.show()\n'

### **Decision Tree Classifier**

In [ ]:

#initialize the model
d1=DecisionTreeClassifier(criterion='entropy', max_depth=7, max_features=None, min_samples_leaf=1, min_samples_split=2)

#Train the model
d1.fit(x_train, y_train)

#Predict for the test set
d1y_pred=d1.predict(x_test)


acc_d1=accuracy_score(y_true=y_test, y_pred=d1y_pred)
#f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='weighted')
f1_d1=f1_score(y_true=y_test, y_pred=d1y_pred, average='weighted')
wP_d1=precision_score(y_true=y_test, y_pred=d1y_pred, average='weighted')
wR_d1=recall_score(y_true=y_test, y_pred=d1y_pred, average='weighted')
#Calculate fbeta score
#f_r1=fbeta_score(y_test, r1y_pred, beta=0.5, average='micro')

#Calculate confusion matrix
#cm3=confusion_matrix(y_test, r1y_pred, labels=observed_emotions).astype(np.float32)

#Print the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_d1*100))
print("d1 for MLP : {:.2f}%".format(f1_d1*100))
print("wp for MLP : {:.2f}%".format(wP_d1*100))
print("wr for MLP : {:.2f}%".format(wR_d1*100))

'''
#Display confusion matrix
plt.imshow(cm4, cmap="binary")
plt.show()
'''

Accuracy for MLP : 47.62%
d1 for MLP : 47.31%
wp for MLP : 47.30%
wr for MLP : 47.62%


'\n#Display confusion matrix\nplt.imshow(cm4, cmap="binary")\nplt.show()\n'